In [ ]:


#@title Indrajith Video Converter with Progress Tracking (Quality Optimized)
#@markdown ### Encoder Options:
#@markdown - **h264_nvenc**: GPU accelerated, 10-20x realtime speed, good for quick processing
#@markdown - **libx264**: CPU based, 1-4x speed, better compression than NVENC
#@markdown - **libx265**: CPU based, 0.3-1x speed, best compression but very slow
#@markdown - **hevc_nvenc**: GPU accelerated, 3-8x speed, good compression balance

#@markdown ---
#@markdown ### File Settings:
input_file = "/content/drive/MyDrive/source.mp4" #@param {type:"string"}
output_file = "/content/drive/MyDrive/OnaHuptak/output.mp4" #@param {type:"string"}

#@markdown ---
#@markdown ### Resolution Settings:
#@markdown Choose output resolution or set custom dimensions
resolution_preset = "Custom" #@param ["720x1280 (Portrait)", "960x1280 (Portrait)", "1080x1920 (Portrait)", "1280x720 (Landscape)", "1920x1080 (Landscape)", "Custom"]
custom_width = 720 #@param {type:"integer"}
custom_height = 960 #@param {type:"integer"}

#@markdown ---
#@markdown ### Quality Settings:
#@markdown Choose encoding mode and quality parameters
encoding_mode = "bitrate" #@param ["crf", "bitrate"]
encoder = "h264_nvenc" #@param ["h264_nvenc", "libx264", "libx265", "hevc_nvenc"]
#@markdown Lower CRF = higher quality/larger file. Higher CRF = lower quality/smaller file
crf_value = 29 #@param {type:"slider", min:15, max:35, step:1}
#@markdown Bitrate settings (only used when encoding_mode = "bitrate")
target_bitrate = "2200k" #@param {type:"string"}
max_bitrate = "2500k" #@param {type:"string"}
buffer_size = "8000k" #@param {type:"string"}

#@markdown ---
#@markdown ### Encoding Speed vs Quality:
#@markdown Slower presets = better quality but longer encoding time
preset_speed = "medium" #@param ["ultrafast", "superfast", "veryfast", "faster", "fast", "medium", "slow", "slower", "veryslow"]

#@markdown ---
#@markdown ### Advanced Quality Options:
scaling_algorithm = "lanczos" #@param ["bilinear", "bicubic", "lanczos", "spline"]
tune_setting = "film" #@param ["film", "animation", "grain", "stillimage", "fastdecode", "zerolatency", "none"]
enable_advanced_quality = True #@param {type:"boolean"}

#@markdown ---
#@markdown ### Watermark Settings:
enable_watermark = False #@param {type:"boolean"}
watermark_text = "Indrajith Gamage" #@param {type:"string"}
watermark_interval_minutes = 1 #@param {type:"integer"}
watermark_duration_seconds = 10 #@param {type:"integer"}

#@markdown ---
#@markdown ### Processing Options:
show_progress = True #@param {type:"boolean"}
show_file_comparison = True #@param {type:"boolean"}
use_sri_lanka_time = True #@param {type:"boolean"}

import subprocess
import time
import os
from datetime import datetime, timedelta

def get_local_time():
    if use_sri_lanka_time:
        from datetime import timezone
        return datetime.now(timezone.utc) + timedelta(hours=5.5)
    else:
        return datetime.now()

def get_resolution():
    if resolution_preset == "Custom":
        return f"{custom_width}x{custom_height}"
    else:
        return resolution_preset.split()[0]

def get_scaling_filter():
    if resolution_preset == "Custom":
        width, height = custom_width, custom_height
    else:
        width, height = resolution_preset.split()[0].split('x')

    return f"scale={width}:{height}:flags={scaling_algorithm}:force_original_aspect_ratio=decrease:force_divisible_by=2"

def build_ffmpeg_command():
    # Get resolution dimensions for watermark font sizing
    if resolution_preset == "Custom":
        width, height = custom_width, custom_height
    else:
        width, height = map(int, resolution_preset.split()[0].split('x'))

    # Build video filter chain
    video_filters = []

    # Add scaling with high-quality algorithm
    scale_filter = get_scaling_filter()
    video_filters.append(scale_filter)

    # Add watermark if enabled
    if enable_watermark:
        font_size = int(height) // 40  # 1/40th of frame height
        interval_sec = watermark_interval_minutes * 60

        # Simplified watermark: appears for duration_seconds every interval_minutes, sliding from right to left
        watermark_filter = f"drawtext=text='{watermark_text}':fontsize={font_size}:fontcolor=white@0.8:x='if(mod(t\\,{interval_sec})<{watermark_duration_seconds}\\,w-w*mod(t\\,{watermark_duration_seconds})/{watermark_duration_seconds}\\,-w)':y=h-th-10"
        video_filters.append(watermark_filter)

    # Combine all video filters
    video_filter_string = ','.join(video_filters)

    # Build base command
    cmd = [
        'ffmpeg', '-y',
        '-i', input_file,
        '-vf', video_filter_string,
        '-c:v', encoder,
        '-c:a', 'copy'
    ]

    # Add encoding parameters based on encoder type and mode
    if encoder in ["h264_nvenc", "hevc_nvenc"]:
        # NVIDIA GPU encoder settings
        if encoding_mode == "crf":
            cmd.extend([
                '-rc', 'vbr',
                '-cq', str(crf_value),
                '-qmin', '15',
                '-qmax', '35',
                '-preset', preset_speed if preset_speed in ["fast", "medium", "slow"] else "medium",
                '-profile:v', 'high',
                '-level', '4.1',
                '-pix_fmt', 'yuv420p'
            ])
        else:  # bitrate mode
            cmd.extend([
                '-rc', 'vbr',
                '-b:v', target_bitrate,
                '-maxrate', max_bitrate,
                '-bufsize', buffer_size,
                '-preset', preset_speed if preset_speed in ["fast", "medium", "slow"] else "medium",
                '-profile:v', 'high',
                '-level', '4.1',
                '-pix_fmt', 'yuv420p'
            ])

        # Add quality enhancement for NVENC
        if enable_advanced_quality:
            cmd.extend(['-spatial-aq', '1', '-aq-strength', '8'])

    else:
        # Software encoder settings (libx264, libx265)
        if encoding_mode == "crf":
            cmd.extend([
                '-crf', str(crf_value),
                '-preset', preset_speed,
                '-profile:v', 'high',
                '-level', '4.1',
                '-pix_fmt', 'yuv420p'
            ])
        else:  # bitrate mode
            cmd.extend([
                '-b:v', target_bitrate,
                '-maxrate', max_bitrate,
                '-bufsize', buffer_size,
                '-preset', preset_speed,
                '-profile:v', 'high',
                '-level', '4.1',
                '-pix_fmt', 'yuv420p'
            ])

        # Add tune setting if not "none"
        if tune_setting != "none":
            cmd.extend(['-tune', tune_setting])

        # Add advanced quality settings for software encoders
        if enable_advanced_quality:
            if encoder == "libx264":
                cmd.extend(['-x264-params', 'me=umh:subme=8:ref=5:bframes=3:direct=auto:weightb=1:mixed-refs=1:8x8dct=1:trellis=2:fast-pskip=0'])
            elif encoder == "libx265":
                cmd.extend(['-x265-params', 'me=3:subme=3:ref=4:bframes=4:rd=4'])

    cmd.append(output_file)
    return cmd

# Start processing
start_time = time.time()
start_local_time = get_local_time()

from google.colab import drive
drive.mount("/content/drive")

print("="*70)
print("🎬 ENHANCED VIDEO CONVERTER STARTING")
print("="*70)
print(f"Started at: {start_local_time.strftime('%Y-%m-%d %H:%M:%S')} {'(Sri Lanka time)' if use_sri_lanka_time else ''}")
print(f"Input file: {input_file}")
print(f"Output file: {output_file}")
print(f"Resolution: {get_resolution()}")
print(f"Encoder: {encoder}")
print(f"Preset: {preset_speed}")
print(f"Scaling: {scaling_algorithm}")
if encoding_mode == "crf":
    print(f"Quality: CRF {crf_value}")
else:
    print(f"Bitrate: {target_bitrate} (max: {max_bitrate})")
if tune_setting != "none":
    print(f"Tune: {tune_setting}")
if enable_advanced_quality:
    print("Advanced quality optimizations: ENABLED")
if enable_watermark:
    print(f"Watermark: '{watermark_text}' (every {watermark_interval_minutes}min for {watermark_duration_seconds}s)")
print("-" * 70)

# Check if input file exists
if not os.path.exists(input_file):
    print(f"❌ ERROR: Input file not found: {input_file}")
    print("Please check the file path and try again.")
else:
    # Build and run command
    cmd = build_ffmpeg_command()

    # Print command for debugging (optional)
    print("FFmpeg command:")
    print(" ".join(cmd))
    print("-" * 70)

    if show_progress:
        print("🚀 Processing video with enhanced quality settings...")
        process = subprocess.Popen(cmd, stderr=subprocess.PIPE, universal_newlines=True)

        while True:
            output = process.stderr.readline()
            if output == '' and process.poll() is not None:
                break
            if output:
                if 'time=' in output or 'frame=' in output:
                    print(f"\r{output.strip()}", end='', flush=True)
                elif 'Duration:' in output:
                    print(f"{output.strip()}")

        return_code = process.poll()
    else:
        print("🚀 Processing video (no progress display)...")
        result = subprocess.run(cmd, capture_output=True, text=True)
        return_code = result.returncode
        if return_code != 0:
            print(f"Error output: {result.stderr}")

    # Calculate completion time
    end_time = time.time()
    end_local_time = get_local_time()
    total_duration = end_time - start_time

    print(f"\n{'='*70}")

    if return_code == 0:
        print("✅ VIDEO CONVERSION COMPLETED SUCCESSFULLY!")
    else:
        print("❌ VIDEO CONVERSION FAILED!")

    print(f"Finished at: {end_local_time.strftime('%Y-%m-%d %H:%M:%S')} {'(Sri Lanka time)' if use_sri_lanka_time else ''}")
    print(f"Total processing time: {total_duration/60:.2f} minutes ({total_duration:.1f} seconds)")

    # File size comparison
    if show_file_comparison and os.path.exists(output_file) and return_code == 0:
        print("-" * 70)
        original_size = os.path.getsize(input_file) / (1024**3)
        new_size = os.path.getsize(output_file) / (1024**3)

        # Get bitrate information
        try:
            # Get original bitrate
            result_orig = subprocess.run(['ffprobe', '-v', 'quiet', '-show_entries', 'format=bit_rate', '-of', 'csv=p=0', input_file], capture_output=True, text=True)
            orig_bitrate = int(result_orig.stdout.strip()) if result_orig.stdout.strip() else 0

            # Get new bitrate
            result_new = subprocess.run(['ffprobe', '-v', 'quiet', '-show_entries', 'format=bit_rate', '-of', 'csv=p=0', output_file], capture_output=True, text=True)
            new_bitrate = int(result_new.stdout.strip()) if result_new.stdout.strip() else 0

            print(f"📁 FILE COMPARISON:")
            print(f"Original: {original_size:.2f} GB @ {orig_bitrate/1000:.0f} kb/s")
            print(f"Processed: {new_size:.2f} GB @ {new_bitrate/1000:.0f} kb/s")
            print(f"Size ratio: {new_size/original_size:.2f}x")
            print(f"Bitrate ratio: {new_bitrate/orig_bitrate:.2f}x" if orig_bitrate > 0 else "Bitrate ratio: N/A")

        except:
            print(f"📁 FILE SIZE COMPARISON:")
            print(f"Original file: {original_size:.2f} GB")
            print(f"Processed file: {new_size:.2f} GB")
            print(f"Size ratio: {new_size/original_size:.2f}x")

        if new_size < original_size:
            savings = ((original_size - new_size) / original_size) * 100
            print(f"Space saved: {savings:.1f}% ({original_size - new_size:.2f} GB)")
        elif new_size > original_size:
            increase = ((new_size - original_size) / original_size) * 100
            print(f"Size increased: {increase:.1f}% (+{new_size - original_size:.2f} GB)")

    print("="*70)